In [1]:
import os
os.chdir('../')

In [2]:
import pandas as pd

df = pd.read_csv('./artifacts/data_ingestion/winequality-red.csv')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [3]:
df.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir: Path
    unzip_data_dir: Path
    STATUS_FILE: str
    all_schema: dict


In [5]:
from src.mlflow_demo.constants import *
from src.mlflow_demo.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, 
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH) -> None:
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        create_directories([self.config.data_validation.root_dir])
        return DataValidationConfig(
            root_dir=self.config.data_validation.root_dir,
            unzip_data_dir=self.config.data_validation.unzip_data_dir,
            STATUS_FILE=self.config.data_validation.STATUS_FILE,
            all_schema=self.schema.COLUMNS
        )
    
    

In [12]:
import os
from src.mlflow_demo import logger

class DataValidation:
    def __init__(self, config: DataValidationConfig) -> None:
        self.config = config

    def validate_all_columns(self) -> bool:
        validation_status = None
        
        try:
            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)
            all_schema = self.config.all_schema.keys()

            missing_columns = [col for col in all_cols if not col in all_schema]

            validation_status = True if len(missing_columns) == 0 else False

            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f'Validation status: {validation_status}')

            return validation_status
        except Exception as e:
            logger.exception(e)
            raise e

In [13]:
try:
    manager = ConfigurationManager()
    config = manager.get_data_validation_config()
    data_validation = DataValidation(config=config)
    data_validation.validate_all_columns()
except Exception as e:
    logger.exception(e)
    raise e

[2023-07-30 21:06:26,701: INFO: common: Yaml config/config.yml loaded successfully.]
[2023-07-30 21:06:26,705: INFO: common: Yaml params.yml loaded successfully.]
[2023-07-30 21:06:26,709: INFO: common: Yaml schema.yml loaded successfully.]
[2023-07-30 21:06:26,711: INFO: common: created directory: artifacts]
[2023-07-30 21:06:26,712: INFO: common: created directory: artifacts/data_validation]
